In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.integrate import quad
from math import log, sqrt, exp

import warnings
warnings.filterwarnings("ignore")

In [2]:
def SABR(F, K, T, alpha, beta, rho, nu):
    X = K
    # if K is at-the-money-forward
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta)**2)/24)*alpha*alpha/(F**(2 - 2*beta))
        numer2 = 0.25*rho*beta*nu*alpha/(F**(1 - beta))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        VolAtm = alpha*(1 + (numer1 + numer2 + numer3)*T)/(F**(1-beta))
        sabrsigma = VolAtm
    else:
        z = (nu/alpha)*((F*X)**(0.5*(1-beta)))*np.log(F/X)
        zhi = np.log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
        numer1 = (((1 - beta)**2)/24)*((alpha*alpha)/((F*X)**(1 - beta)))
        numer2 = 0.25*rho*beta*nu*alpha/((F*X)**((1 - beta)/2))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        numer = alpha*(1 + (numer1 + numer2 + numer3)*T)*z
        denom1 = ((1 - beta)**2/24)*(np.log(F/X))**2
        denom2 = (((1 - beta)**4)/1920)*((np.log(F/X))**4)
        denom = ((F*X)**((1 - beta)/2))*(1 + denom1 + denom2)*zhi
        sabrsigma = numer/denom

    return sabrsigma

In [3]:
def IRR_0(K, m, N):
    # implementation of IRR(K) function
    value = 1/K * ( 1.0 - 1/(1 + K/m)**(N*m) )
    return value

def IRR_1(K, m, N):
    # implementation of IRR'(K) function (1st derivative)
    firstDerivative = -1/K*IRR_0(K, m, N) + 1/(K*m)*N*m/(1+K/m)**(N*m+1)
    return firstDerivative

def IRR_2(K, m, N):
    # implementation of IRR''(K) function (2nd derivative)
    secondDerivative = -2/K*IRR_1(K, m, N) - 1/(K*m*m)*(N*m)*(N*m+1)/(1+K/m)**(N*m+2)
    return secondDerivative


## Question 1: Decompounded Option

In [4]:
def g_0(K):
    return F ** (1/4) - 0.04**(1/2)

def g_1(K):
    return (1/4) * K** (-3/4)

def g_2(K):
    return (-3/16)*(K)**(-7/4)

def h_0(K, m, N):
    # implementation of h(K)
    value = g_0(K) / IRR_0(K, m, N)
    return value

def h_1(K, m, N):
    # implementation of h'(K) (1st derivative)
    firstDerivative = (IRR_0(K, m, N)*g_1(K) - g_0(K)*IRR_1(K, m, N)) / IRR_0(K, m, N)**2
    return firstDerivative

def h_2(K, m, N):
    # implementation of h''(K) (2nd derivative)
    secondDerivative = ((IRR_0(K, m, N)*g_2(K) - IRR_2(K, m, N)*g_0(K) - 2.0*IRR_1(K, m, N)*g_1(K))/IRR_0(K, m, N)**2 
                        + 2.0*IRR_1(K, m, N)**2*g_0(K)/IRR_0(K, m, N)**3)
    return secondDerivative


In [5]:
def Black76Call(F, K, sigma, T):
    d1 = (log(F / K) + (1 / 2) * (sigma ** 2) * T) / (sigma * sqrt(T))
    d2 = d1 - sigma * sqrt(T)
    V =  F * norm.cdf(d1) - K * norm.cdf(d2)
    return V

def Black76Put(F, K, sigma, T):
    d1 = (log(F / K) + (1 / 2) * (sigma ** 2) * T) / (sigma * sqrt(T))
    d2 = d1 - sigma * sqrt(T)
    V = K * norm.cdf(-d2) - F * norm.cdf(-d1)
    return V

In [6]:
def integrand_put(K, m, N, sigma, df, F, T):
    h = h_2(K, m, N)
    V_put = df * Black76Put(F, K, sigma, T)
    return h * V_put

def integrand_call(K, m, N, sigma, df, F, T):
    h = h_2(K, m, N)
    V_call = df * Black76Call(F, K, sigma, T)
    return h * V_call

In [7]:
disc_factor = pd.read_csv('Discount_Factors.csv') 
disc_factor.set_index("Tenor",inplace = True)

swap_rate_df = pd.read_csv('forward_swap_rate.csv')

Alpha_SABR = pd.read_csv('Alpha_SABR.csv')
Rho_SABR = pd.read_csv('Rho_SABR.csv')
Nu_SABR = pd.read_csv('Nu_SABR.csv')

In [8]:
## using implicit index on former dataframe o call out F, alpha, beta, rho and nu, if former df change, might lead to problem 

F = swap_rate_df.iloc[4,2]  # 5y * 10 y forward swap rate
DF_5y = disc_factor.loc[5.0,"OIS_DF"] # OIS D(0,5y)
T = 5 # expiry
alpha = Alpha_SABR.iloc[1,-1]  # 5y * 10 y alpha form SABR calibarition result
beta = 0.9 
rho = Rho_SABR.iloc[1,-1]  # 5y * 10 y rho form SABR calibarition result
nu =  Nu_SABR.iloc[1,-1]  # 5y * 10 y nu form SABR calibarition result
m = 2 # payment frequency
N = 10 # tenor

integral_term1 = quad(lambda x: integrand_put(x, m, N, SABR(F, x, T, alpha, beta, rho, nu), IRR_0(F, N, m) , F, T), 0, F)
integral_term2 = quad(lambda x: integrand_call(x, m, N, SABR(F, x, T, alpha, beta, rho, nu), IRR_0(F, N, m), F, T), F, np.inf)

PV = g_0(F)*DF_5y  + integral_term1[0] + integral_term2[0]

In [9]:
print("The PV of the CMS Decompounded Option is", PV)

The PV of the CMS Decompounded Option is 0.24769310685896034


## Question 2: Caplet

In [10]:
L = 0.04**((1/2)*4)

sigma = SABR(F, L, T, alpha, beta, rho, nu)
V_call = DF_5y * IRR_0(F, m, N) * Black76Call(F, L, sigma, T)

part1 = h_1(L, m, N)* V_call
part2 = quad(lambda x: h_2(x, m, N)*IRR_0(F, m, N)*Black76Call(F, x, SABR(F, x, T, alpha, beta, rho, nu), T),L,np.inf)
# part2 = quad(lambda x: integrand_call(x, m, N, SABR(F, x, T, alpha, beta, rho, nu), IRR_0(F, N, m), F, T), L, np.inf)

PV_caplet = part1 + part2[0]

print("The PV of the CMS Decompounded Option Caplet is", PV_caplet)


The PV of the CMS Decompounded Option Caplet is 0.276157104971118
